In [2]:
import json
tweets = []
for line in open('tweets.json', 'r'):
    tweets.append(json.loads(line))

In [3]:
# tweets[0]
len(tweets)
tweets[0:5]

[{'in_reply_to_status_id_str': None,
  'text': 'If mitt romney say he gunna legalize weed then obama is fucked',
  'geo': None,
  'in_reply_to_screen_name': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'id_str': '246058205367644160',
  'retweeted': False,
  'created_at': 'Thu Sep 13 01:30:22 +0000 2012',
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'entities': {'urls': [], 'user_mentions': [], 'hashtags': []},
  'in_reply_to_status_id': None,
  'place': None,
  'contributors': None,
  'coordinates': None,
  'favorited': False,
  'user': {'follow_request_sent': None,
   'statuses_count': 23815,
   'profile_background_image_url_https': 'https://si0.twimg.com/profile_background_images/643937019/07gmvz818szy9m7tupnt.jpeg',
   'default_profile_image': False,
   'friends_count': 721,
   'profile_sidebar_border_color': '000000',
   'id_str': '237158178',
   'is_translator': False,
   'verified': False,
   'fav

In [8]:
import pandas as pd
# tweets_df = pd.DataFrame(columns=['id','tweet'],index=None)
for tweet in tweets[100000:200000]:
    data = pd.DataFrame({"id":[tweet['id_str']],"tweet":[tweet['text']]})
    tweets_df = tweets_df.append(data, ignore_index = True)

In [9]:
len(tweets_df)

200000

In [ ]:
tweets_df.iloc[1]['tweet']
print(len(tweets_df))

In [ ]:
# import re
# x = tweets_df.iloc[1]['tweet']
# x = "@SuePalmers As a youth who has lived abroad, I share the President's feelings. It changes how you view foreign relations quite a bit. #election2012"
# tweets_df.iloc[1]['tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([# \t])|(\w+:\/\/\S+)"," ",x).split())

In [ ]:
import re
for i in range (0,1000000):
#     print(i)
    x = tweets_df.iloc[i]['tweet']
#     tweets_df.iloc[i]['tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([# \t])|(\w+:\/\/\S+)"," ",x).split())
    tweets_df.iloc[i]['tweet'] = ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

In [ ]:
# z = "RT @SuePalmers As a youth who has lived abroad, I share the President's feelings. It changes how you view foreign relations quite a bit."
# y = ' '.join(re.sub("(^RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",z).split())
# print(y)

In [ ]:
# for i in range (0,10):
# #     for word in tweet['text']:
#         print(tweets_df.iloc[i]['tweet'])

In [ ]:
for i in range (0,11):
#     for word in tweet['text']:
        print(tweets_df.iloc[i]['tweet'])

In [ ]:
tweets_df.iloc[1]['tweet']

In [ ]:
from nltk.tokenize import word_tokenize
# text = word_tokenize("And now for something completely different")

In [ ]:
# import nltk
# text = word_tokenize(tweets_df.iloc[1]['tweet'])
# temp = nltk.pos_tag(text)

In [ ]:
import nltk
tweets_imp_words_df = pd.DataFrame(columns=['id','Adj_Adv_Verb','pos'],index=None)
for i in range (0,1000000):
    text = word_tokenize(tweets_df.iloc[i]['tweet'])
    pos_tagged_words = nltk.pos_tag(text)
    tempStr = ''
    pos = []
    for i in range(0,len(pos_tagged_words)):
        if pos_tagged_words[i][1] in ['JJ','JJR','JJS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ']:
            tempStr += pos_tagged_words[i][0]+" "
            pos.append(pos_tagged_words[i][1])
    tempData = pd.DataFrame({"id":[tweets_df.iloc[i]['id']],"Adj_Adv_Verb":[tempStr],"pos":[pos]})
    tweets_imp_words_df = tweets_imp_words_df.append(tempData, ignore_index = True)

In [ ]:
tweets_imp_words_df.head()
print(len(tweets_imp_words_df))

In [ ]:
# temp

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

sid = SentimentIntensityAnalyzer()
individual_score = []
for i in range (0,1000000):
    temp_individual_score = []
    for word in tweets_imp_words_df.iloc[i]['Adj_Adv_Verb'].split():
        temp_individual_score.append(sid.polarity_scores(word)['compound'])
    individual_score.append(temp_individual_score)
tweets_imp_words_df['individual_scores'] = individual_score

In [ ]:
tweets_imp_words_df.head()

In [ ]:
print(tweets_imp_words_df.iloc[10]['Adj_Adv_Verb'])
print(tweets_imp_words_df.iloc[10]['individual_scores'])

In [ ]:
for i in range (0,1000000):
    pos = tweets_imp_words_df.iloc[i]['pos']
    for j in range (1,len(tweets_imp_words_df.iloc[i]['individual_scores'])):
        if(pos[j] in ['JJ','JJR','JJS'] and pos[j-1] not in ['JJ','JJR','JJS']):
            if(tweets_imp_words_df.iloc[i]['individual_scores'][j-1] > 0):
                tweets_imp_words_df.iloc[i]['individual_scores'][j-1] *= tweets_imp_words_df.iloc[i]['individual_scores'][j]
            elif(tweets_imp_words_df.iloc[i]['individual_scores'][j-1] < 0):
                tweets_imp_words_df.iloc[i]['individual_scores'][j] = 5 - tweets_imp_words_df.iloc[i]['individual_scores'][j]

In [ ]:
tweets_imp_words_df.iloc[0:10]

In [ ]:
score = []
for i in range (0,1000000):
    temp_score = 0;
    no_of_adj = 0;
    pos = tweets_imp_words_df.iloc[i]['pos']
    for j in range (0,len(tweets_imp_words_df.iloc[i]['individual_scores'])):
        temp_score += tweets_imp_words_df.iloc[i]['individual_scores'][j]
        if(pos[j] in ['JJ','JJR','JJS']):
            no_of_adj += 1;
    if no_of_adj > 0:
        temp_score = temp_score/no_of_adj;
    score.append(temp_score)
tweets_imp_words_df['score'] = score

In [ ]:
tweets_imp_words_df[0:19]

In [ ]:
pos = 0;
neg = 0;
neu = 0;
for i in range (0,1000000):
    if tweets_imp_words_df.iloc[i]['score'] > 0:
        pos += 1
    elif  tweets_imp_words_df.iloc[i]['score'] < 0:
        neg += 1
    else:
        neu += 1

In [ ]:
print(pos)
print(neg)
print(neu)

In [64]:
is_obama = []
is_romney = []
for i in range(0,10):
    obama = False;
    romney = False;
    for word in tweets_df.iloc[i]['tweet'].split():
        print(word)
        if word.lower() in ["obama","barack"]:
            obama = True
        if word.lower() in["mitt","romney"]:
            romney = True
    is_obama.append(obama)
    is_romney.append(romney)
tweets_imp_words_df['is_obama'] = is_obama
tweets_imp_words_df['is_romney'] = is_romney

If
mitt
romney
say
he
gunna
legalize
weed
then
obama
is
fucked
As
a
youth
who
has
lived
abroad,
I
share
the
President's
feelings.
It
changes
how
you
view
foreign
relations
quite
a
bit.
RT
:
This
"President"
has
the
gall
to
apologize
for
our
way
of
life?.
You
cant
get
Americans
jobs,
but
you
wanna
make
Musl
...
Can
we
please
stop
pretending
that
Obama
is
a
good
man?
He
is
not.
Today
proved
that.
Being
a
good
father
alone
does
not
make
you
a
good
man
RT
:
I
am
glad
Mitt
Romney
defended
America
and
free
speech.
It
is
clear
President
Obama
is
too
busy
appeasing
radicals
to
...
Business
Tax
Accountant
openings
Youngstown
Ohio
USA
RT
:
The
official
GOP
response
to
Americans
being
killed
abroad
yesterday
is
that
Obama
is
on
the
side
of
the
killers.
http:
...
usa
esto
viejjaaa!
Call
Center
Specialist
careers
Vandalia
Illinois
USA
-careers.html
RT
:
Obama
would
never
make
a
good
running
back.
He
keeps
dropping
his
balls.


In [65]:
tweets_imp_words_df

,id,Adj_Adv_Verb,pos,individual_scores,score,is_obama,is_romney
0,246058208366559232,mitt say gunna then obama is fucked,"[JJ, VBP, VBZ, RB, VBZ, VBZ, VBN]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.6597]",-0.659700,True,True
1,246058213500407808,has lived abroad changes view foreign quite,"[VBZ, VBN, RB, VBZ, VBP, JJ, RB]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,False,False
2,246058214507048960,has apologize cant get wan na make,"[VBZ, VB, VBP, VB, VBP, JJ, VBP]","[0.0, 0.1027, 0.0, 0.0, 0.0, 0.0, 0.0]",0.102700,False,False
3,246058214158897152,please stop is good is not proved Being good a...,"[VB, VB, VBZ, JJ, VBZ, RB, VBD, VBG, JJ, RB, V...","[0.3182, -0.296, 0.0, 0.4404, 0.0, 0.0, 0.0, 0...",0.364467,True,False
4,246058214276354050,am glad defended free is clear is too busy app...,"[VBP, JJ, VBD, JJ, VBZ, JJ, VBZ, RB, JJ, VBG]","[0.0, 0.4588, 0.0, 0.5106, 0.0, 0.3818, 0.0, 0...",0.400300,True,True
5,246058207460614144,,[],[],0.000000,False,False
6,246058213500407808,official being killed abroad is is,"[JJ, VBG, VBN, RB, VBZ, VBZ]","[0.0, 0.0, -0.6705, 0.0, 0.0, 0.0]",-0.670500,True,False
7,246058206651117568,usa,[JJ],[0.0],0.000000,False,False
8,246058208584683521,Call,[VB],[0.0],0.000000,False,False
9,246058212435038208,never make good back keeps dropping,"[RB, VB, JJ, RB, VBZ, VBG]","[0.0, 0.0, 0.4404, 0.0, 0.0, 0.0]",0.440400,True,False


In [ ]:
print("hello")